In [1]:
import numpy as np
import pandas as pd
import os

In [13]:
from bokeh.io import show, output_notebook
output_notebook()
from bokeh.models import LogColorMapper, ColumnDataSource, HoverTool
from bokeh.palettes import Viridis6 as palette
from bokeh.plotting import figure
from bokeh.tile_providers import CARTODBPOSITRON

Loading BokehJS ...

In [3]:
data_dir = '../boat/data/'
logs = os.listdir(data_dir)
dfs= [pd.read_csv(data_dir + fname, sep='\t') for fname in logs]
merged = pd.concat(dfs)

In [4]:
merged.head()

,boat_COG,boat_heading,boat_lat,boat_lon,datetime,skiff_COG,skiff_heading,skiff_lat,skiff_lon
0,NaN,233.128609,61.077967,-146.568223,2018-07-24 16:10:09,NaN,NaN,NaN,NaN
1,NaN,232.160508,61.077972,-146.568230,2018-07-24 16:10:14,NaN,NaN,NaN,NaN
2,NaN,231.948268,61.077972,-146.568230,2018-07-24 16:10:14,NaN,NaN,NaN,NaN
3,NaN,231.768254,61.077972,-146.568230,2018-07-24 16:10:14,NaN,NaN,NaN,NaN
4,157.56,232.567460,61.077980,-146.568215,2018-07-24 16:10:29,NaN,NaN,NaN,NaN


In [6]:
def default_map():
    p = figure(x_range=(lon.min(), lon.max()), 
           y_range=(lat.min(), lat.max()), 
           x_axis_type="mercator", 
           y_axis_type="mercator",
           tools="pan,wheel_zoom,reset,zoom_in,zoom_out",
           active_scroll='wheel_zoom')
    p.add_tile(CARTODBPOSITRON)
    return p

In [22]:
def to_WM(lat, lon):
    '''convert "normal" WGS84 coordinates to the web mercator coordinates that bokeh uses
    code copied from 
    https://gis.stackexchange.com/questions/247871/convert-gps-coordinates-to-web-mercator-epsg3857-using-python-pyproj
    '''
    
    # derived from the Java version explained here: http://wiki.openstreetmap.org/wiki/Mercator
    RADIUS = 6378137.0 # in meters on the equator

    lat2 = np.log(np.tan(np.pi / 4 + np.radians(lat) / 2)) * RADIUS
    lon2 = np.radians(lon) * RADIUS
    return lat2, lon2

lat, lon = to_WM(merged['boat_lat'], merged['boat_lon'])
merged['boat_lat_WM'] = lat
merged['boat_lon_WM'] = lon

my_hover = HoverTool()
my_hover.tooltips = [('Time:', '@datetime')]

p = figure(x_range=(lon.min(), lon.max()), 
           y_range=(lat.min(), lat.max()), 
           x_axis_type="mercator", 
           y_axis_type="mercator",
           tools="pan,wheel_zoom,reset,zoom_in,zoom_out",
           active_scroll='wheel_zoom')
p.add_tile(CARTODBPOSITRON)
p.line(x='boat_lon_WM', y='boat_lat_WM', source=merged)
p.circle(x='boat_lon_WM', y='boat_lat_WM', source=merged)
p.add_tools(my_hover)
show(p)

In [11]:
from bokeh.models import Arrow, OpenHead, NormalHead, VeeHead
p = figure(plot_width=600, plot_height=600)

p.circle(x=[0, 1, 0.5], y=[0, 0, 0.7], radius=0.1,
         color=["navy", "yellow", "red"], fill_alpha=0.1)

p.add_layout(Arrow(end=OpenHead(line_color="firebrick", line_width=4),
                   x_start=0, y_start=0, x_end=1, y_end=0))

p.add_layout(Arrow(end=NormalHead(fill_color="orange"),
                   x_start=1, y_start=0, x_end=0.5, y_end=0.7))

p.add_layout(Arrow(end=VeeHead(size=35), line_color="red",
                   x_start=.0001, y_start=.0001, x_end=0, y_end=0))

show(p)